In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import json
import logging
import os
import re
import sys
from functools import partial
from itertools import chain
from pathlib import Path
from uuid import uuid4

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser

from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root
)
pd.options.plotting.backend = "plotly"

In [4]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-02-08 16:10:34,284 - Authenticating...
2023-02-08 16:10:34,843 - Successfully authenticated


In [5]:
super_clarion(clarion)

In [6]:
clarion.metrics()

{'risk_metrics':            metricType assetClass  \
 0      IntrinsicValue        Any   
 1   LocalPresentValue        Any   
 2         OptionTheta        Any   
 3         OptionTheta        Any   
 4         OptionTheta        Any   
 5        PresentValue        Any   
 6               Theta        Any   
 7           TimeValue        Any   
 8     CorrelationRisk     Credit   
 9                Cs01     Credit   
 10              Delta     Credit   
 11              Gamma     Credit   
 12               Vega     Credit   
 13              Delta     Equity   
 14       DeltaPercent     Equity   
 15      DividendDelta     Equity   
 16              Gamma     Equity   
 17                Rho     Equity   
 18               Spot     Equity   
 19              Vanna     Equity   
 20               Vega     Equity   
 21              Volga     Equity   
 22              Carry         Fx   
 23              Carry         Fx   
 24              Delta         Fx   
 25              Delta

In [7]:
pd.concat(clarion.metrics()).to_clipboard()

In [8]:
pv = clarion.metric('pv', 'PresentValue', 'Any')
local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any')
strike = clarion.metric('strike', 'Strike', 'Any')
notional = clarion.metric('notional', 'Notional', 'Any')
expiry = clarion.metric('expiry', 'ExpiryDate', 'Any')
maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'})
start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'})
fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates')
delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'})
theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'})
price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'})
price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'})
spot = clarion.metric('spot', 'Spot', 'Rates')
vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'})
atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'})
atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'})
strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'})
strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'})
carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'})

metrics = [
    pv,
    local_pv,
    strike,
    notional,
    expiry,
    maturity,
    start,
    fwd_rate,
    spot,
    delta,
    theta,
    price_cents,
    price_bps,
    # vega,
    # atm_vol_day,
    atm_vol_ann,
    # strike_vol_day,
    strike_vol_ann,
    # carry_day
]

### EUR BEAR FLAT (BUY structure @ 2m)

In [9]:
trades = dict(
    eur_9m_bear_flat = dict(
        t1 = clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 3M',
            expiry='9m',
            tenor='1y',
            strike=3.65,
            swaption_type='Pay',
            notional=2e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Fwd',
            #id='eur_9m_bear_flat'
        ),

        t2 = clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='9m',
            tenor='2y',
            strike=3.4,
            swaption_type='Pay',
            notional=-1e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Fwd',
            #id='eur_9m_bear_flat'
        ),

        t3 = clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='9m',
            tenor='2y',
            strike=2.8,
            swaption_type='Rec',
            notional=1e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Fwd',
            #id='eur_9m_bear_flat'
        ),

        t4 = clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='9m',
            tenor='2y',
            strike=2.0,
            swaption_type='Rec',
            notional=-1e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Fwd',
            #id='eur_9m_bear_flat'
        ),
    ),
    eur_15m_bull_steep = [
            clarion.instruments.RatesSwaption(
                currency='EUR',
                instrument_name='EURIBOR A 3M',
                expiry='2024-05-03',
                tenor='1y',
                strike=2.425,
                swaption_type='Rec',
                notional=-1e9,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Fwd',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='EUR',
                instrument_name='EURIBOR A 3M',
                expiry='2024-05-03',
                tenor='1yx1y',
                strike=2.0,
                swaption_type='Rec',
                notional=1e9,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Fwd',
                #id='eur_9m_bear_flat'
            ),
    ]
)

In [10]:
sep = '__'

In [11]:
all_trades = []
i = 0
for key, val in trades.items():
    if isinstance(val, dict):
        for key2, trade in val.items():
            trade.id = f"{key}{sep}{key2}"
            all_trades.append(trade)
            i += 1
    elif isinstance(val, list):
        for trade in val:
            trade.id = f"{key}{sep}{i}"
            all_trades.append(trade)
            i += 1

In [12]:
from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millify
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [13]:
runner = Runner(clarion)

In [14]:
df = runner.run(metrics, all_trades)

2023-02-08 16:10:57,194 - Requesting data
2023-02-08 16:11:00,484 - Request complete
2023-02-08 16:11:00,708 - Response Count Summary - DFRAME:6


In [15]:
res = pd.concat([pd.DataFrame(df['id'].str.split(sep).to_list(), columns=['trade', 'id']), runner.df.drop('id', axis=1)], axis=1)
col_order = [*res.columns[~res.columns.isin([metric.label for metric in metrics])], *[metric.label for metric in metrics]]
res['notional'] = res['notional'].astype(float)
res = res[col_order]

In [18]:
ress = []

for index, group in res.groupby('trade'):
    totals = pd.Series(
        {
            'trade': index,
            'id': 'total',
            'price_bps': (group['price_bps'] * np.sign(group['notional'])).sum(),
            'pv': group['pv'].sum(),
            'local_pv': group['local_pv'].sum(),
            'delta': group['delta'].sum(),
            'fwd_rate': (group['fwd_rate'] * np.sign(group['notional'])).sum(),
        }
    ).to_frame().T
    r = pd.concat([totals, group])[col_order]
    ress.append(r)
ress = pd.concat(ress)

In [19]:
ress

trade     id periodType  \
0  eur_15m_bull_steep  total        NaN   
0  eur_15m_bull_steep      4      daily   
1  eur_15m_bull_steep      5      daily   
0    eur_9m_bear_flat  total        NaN   
2    eur_9m_bear_flat     t1      daily   
3    eur_9m_bear_flat     t2      daily   
4    eur_9m_bear_flat     t3      daily   
5    eur_9m_bear_flat     t4      daily   

                                         description     period pv local_pv  \
0                                                NaN        NaT  0        0   
0  EUR EURIBOR A 3M 03-May-24 1y 2.425 Receiver U... 2023-02-08  0        0   
1  EUR EURIBOR A 3M 03-May-24 1yx1y 2 Receiver US... 2023-02-08  0        0   
0                                                NaN        NaT  0        0   
2  EUR EURIBOR A 3M 9m 1y 3.65 Payer USD CashPric... 2023-02-08  0        0   
3  EUR EURIBOR A 6M 9m 2y 3.4 Payer USD CashPrice... 2023-02-08  0        0   
4  EUR EURIBOR A 6M 9m 2y 2.8 Receiver USD CashPr... 2023-02-08  0        0   
5  EUR EURIBOR A 6M 9m 2y 2 Receiver USD CashPric... 2023-02-08  0        0   

   strike  notional      expiry maturity_date  start_date fwd_rate   spot  \
0     NaN       NaN         NaN           NaN         NaN  -0.2795    NaN   
0  2.4250    -1.00b  2024-05-03    2025-05-07  2024-05-07   2.9447 3.3412   
1  2.0000     1.00b  2024-05-03    2026-05-07  2025-05-07   2.6652 3.1179   
0     NaN       NaN         NaN           NaN         NaN   0.2107    NaN   
2  3.6500     2.00b  2023-11-08    2024-11-11  2023-11-10   3.3140 3.3412   
3  3.4000    -1.00b  2023-11-08    2025-11-10  2023-11-10   3.1033 3.3112   
4  2.8000     1.00b  2023-11-08    2025-11-10  2023-11-10   3.1033 3.3112   
5  2.0000    -1.00b  2023-11-08    2025-11-10  2023-11-10   3.1033 3.3112   

    delta   theta  price_cents price_bps  atm_vol_ann  strike_vol_ann  
0   5.80k     NaN          NaN    0.4927          NaN             NaN  
0  29.55k   5.24k      16.3434   17.4466      86.5950         85.2355  
1 -23.75k  -5.62k      16.3965   17.9394     100.3046         97.4194  
0 -61.56k     NaN          NaN   11.1920          NaN             NaN  
2  59.00k -14.79k      15.9130   16.7537      83.9210         88.0746  
3 -62.99k  16.41k      38.6655   20.6689      92.4647         96.0048  
4 -72.91k -15.38k      35.1417   18.7852      92.4647         90.9323  
5  15.35k   7.34k       6.8804    3.6780      92.4647         95.9708

In [ ]:
t = clarion.instruments.RatesSwaption(
    currency='EUR',
    instrument_name='EURIBOR A 3M',
    expiry='15m',
    tenor='1y',
    strike='2.425',
    swaption_type='Rec',
    notional=1000000,
    settle_type='Cash Price',
    collateral_currency='USD',
    pay_settle='Spot'
)

t2 = clarion.instruments.RatesSwaption(
    currency='EUR',
    instrument_name='EURIBOR A 3M',
    expiry='15m',
    tenor='1y',
    strike='2.425',
    swaption_type='Rec',
    notional=1000000,
    settle_type='Cash Price',
    collateral_currency='USD',
    pay_settle='Spot'
)

In [ ]:
fut_opt_exp = {
    'h4': [
        ('g3', 0.25),
        ('h3', 0.25),
        ('h4', 0.25),
    ], 
    'm4': [
        ('j3', 0.0625),
        ('k3', 0.0625),
        ('m3', 0.0625),
        ('m4', 0.125),
    ], 
    'u4': [
        ('n3', 0.0625),
        ('u3', 0.125),
        ('u4', 0.125),
    ],
    'z4': [
        ('z3', 0.125),
        ('z4', 0.125),
    ]
}


In [ ]:
r = clarion.price.run(metrics, data_list=[t.trade], start_date='2020-01-01',  end_date='2023-02-07', market_mode='reval')

In [ ]:
r['D']

In [ ]:
def get_k1(underlying='sfr3', oom=0.25, min_k_inc=0.0625):
    # underlying = 'sfr'
    current = bq.bdp(
        f"{underlying} Comdty",
        'px_last'
    )

    p = current['px_last'].squeeze()
    
    rem = (p + oom) % 1
    k1 = int(p + oom) + np.round(rem / min_k_inc) * min_k_inc
    return p, k1

In [ ]:
ooms = [0.25, 0.50, 0.75]
oom = ooms[0]
year = 2
wide = 0.50 

In [ ]:
get_k1('SFRH4', oom=0.5, min_k_inc=0.25)

In [ ]:
ress = {}

for fut_exp, val in fut_opt_exp.items():
    fut = f"sfr{fut_exp}"
    for opt_exp, min_k_inc in val:
        for oom in ooms:
            p, k1 = get_k1(fut, oom=oom, min_k_inc=min_k_inc)

            opt_exp_ = f"{opt_exp[0]}{year}{opt_exp[1]}".upper()
            fut_exp_ = f"{fut_exp[0]}{year}{fut_exp[1]}".upper()


            opt1 = clarion.instruments.RatesShortTermFutureOption(
                id=f"{opt_exp}x{fut_exp}@{k1-wide} {oom} oom",
                currency='USD',
                instrument_name='SOFR 3M CME',
                expiry=opt_exp_,
                tenor=fut_exp_,
                strike=k1,
                option_type='Call',
                notional=100
            )

            opt2 = clarion.instruments.RatesShortTermFutureOption(
                id=f"{opt_exp}x{fut_exp}@{k1 + wide} {oom} oom",
                currency='USD',
                instrument_name='SOFR 3M CME',
                expiry=opt_exp_,
                tenor=fut_exp_,
                strike=k1+ wide,
                option_type='Call',
                notional=-100
            )
            res = clarion.price.run(
                metrics=metrics,
                data_list=[opt1.trade, opt2.trade],
                market_mode='live'
            )
            
            ress[(fut_exp, opt_exp, oom)] = res['DFRAME']

In [ ]:
ress = {(r[0].lower(), r[1].lower(), r[2]): v for r, v in ress.items()}

In [ ]:
df = pd.concat(ress)

In [ ]:
df = df.rename_axis(['fut_exp', 'opt_exp', 'oom', 'buy_sell'])

In [ ]:
df['signed_price_bps'] = df['price_bps'] * - np.sign(df['theta'])

In [ ]:
df['signed_price_cents'] = df['price_cents'] * - np.sign(df['theta'])

In [ ]:
df['scaled_theta'] = df['theta'] / df['strike_vol_day']

In [ ]:
g = df.reset_index().groupby(['fut_exp', 'opt_exp', 'oom'], sort=False, as_index=False)

In [ ]:
g.sum().to_clipboard()

In [ ]:
df.to_clipboard()

In [ ]:
opt = clarion.instruments.RatesShortTermFutureOption(
    currency='USD',
    instrument_name='SOFR 3M CME',
    expiry=opt_exp_,
    tenor=fut_exp_,
    strike=k1,
    option_type='Call',
    notional=100
)

In [ ]:
opt

In [ ]:
res = clarion.price.run(
    metrics=metrics,
    data_list=[opt.trade],
    market_mode='live'
)

In [ ]:
(96.178 - 96) * 2500 * 100

In [ ]:
df = _['DFRAME']

In [ ]:
df = _['DFRAME']

In [ ]:
df

In [ ]:
fut, p, k1

In [ ]:
clarion.risk_metrics().query('assetClass =="Credit"')

In [ ]:
clarion.price_metrics().query('assetClass =="Credit"')

In [ ]:
# Create a list of metrics in form clarion.metric('label', 'metricType', 'assetClass', {config})
# 'label' here is an arbitrary string that can be used to identify the metric
pv = clarion.metric('pv', 'PresentValue', 'Any')
strike = clarion.metric('strike', 'Strike', 'Any')
maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'})
start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'})

fx_price = clarion.metric('Price', 'Price', 'Fx', {'Unit': 'PercentLocal'}) 
atm_vol = clarion.metric('AtmVol', 'Volatility', 'Fx', {'Type': 'ATM'})
strike_vol = clarion.metric('StrikeVol', 'Volatility', 'Fx', {'Type': 'Strike'})
carry_t1 = clarion.metric('T+1Carry', 'Carry', 'Fx', {'Period': '1', 'StartBumpDays': '1'})


fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates')
strike_vol = clarion.metric('strike_vol', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'})

# m8 = clarion.metric('EqDelta', 'Delta', 'Equity', {'VolSpotDynamic': 'StickyStrike'})
# m9 = clarion.metric('EqPrice', 'Price', 'Equity', {'Unit': 'Local'})

fx_metrics = [pv, fx_price, atm_vol, strike_vol, carry_t1]
rates_metrics = [start, maturity, pv, strike, fwd_rate, strike_vol]
# eq_metrics = [m1,m8,m9]

In [ ]:
clarion.instruments.RatesShortTermFutureOption()

In [ ]:
from gioutils.tools_clarion import get_history_plotter_book

In [ ]:
data, p = get_history_plotter_book(clarion, bq, stir_as_swap=False)

In [ ]:
from gioutils.ezutils import get_irbt_ts
from gioutils.clarion_ext import Runner

In [ ]:
bbg_instruments = [
    'RatesTreasuryFuturesOption',
    'RatesShortFuturesOption',
    'RatesTreasuryFutures',
    'RatesShortFutures'
]

In [ ]:
end_date = today()
offset = "20y"
start_date = end_date - parse_offset(offset)

In [ ]:
def get_swaps_from_postions(data):
    data = data.query('TradeType == "RatesSwap" or as_swap == True')
    
    swaps = []
    for index, swap in data.iterrows():
        
        period = swap['hpperiod'].lower().split('x')
        
        i = clarion.instruments.RatesSwap(
            currency=swap['LocalCurrency'],
            instrument_name=swap['Underlying'],
            strike='a',
            tenor=period[1] if len(period) > 1 else period[0],
            swap_type=swap['PayReceive'][:3],
            start_date=period[0] if len(period) > 1 else None,
            clearing_house='LCH',
            id=str(swap['id'])
        )
        swaps.append(i)
    return swaps

In [ ]:
def get_

In [ ]:
0.125 / 2 

In [ ]:
strike 

In [ ]:
swaps = get_swaps_from_postions(data)

In [ ]:
swaps

In [ ]:
clarion.price_metrics().sort_values('metricType')[100:]

In [ ]:
# Create a list of metrics in form clarion.metric('label', 'metricType', 'assetClass', {config})
# 'label' here is an arbitrary string that can be used to identify the metric
pv = clarion.metric('pv', 'PresentValue', 'Any')
strike = clarion.metric('strike', 'Strike', 'Any')
maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'})
start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'})

fx_price = clarion.metric('Price', 'Price', 'Fx', {'Unit': 'PercentLocal'}) 
atm_vol = clarion.metric('AtmVol', 'Volatility', 'Fx', {'Type': 'ATM'})
strike_vol = clarion.metric('StrikeVol', 'Volatility', 'Fx', {'Type': 'Strike'})
carry_t1 = clarion.metric('T+1Carry', 'Carry', 'Fx', {'Period': '1', 'StartBumpDays': '1'})


fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates')
strike_vol = clarion.metric('strike_vol', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'})

# m8 = clarion.metric('EqDelta', 'Delta', 'Equity', {'VolSpotDynamic': 'StickyStrike'})
# m9 = clarion.metric('EqPrice', 'Price', 'Equity', {'Unit': 'Local'})

fx_metrics = [pv, fx_price, atm_vol, strike_vol, carry_t1]
rates_metrics = [start, maturity, pv, strike, fwd_rate, strike_vol]
# eq_metrics = [m1,m8,m9]

In [ ]:
strike = clarion.metric("strike", "Strike", "Any")

In [ ]:
runner = Runner(clarion=clarion)

In [ ]:
res = runner.run(
    metrics=strike,
    data_list=swaps,
    market_mode="reval",
    start_date=start_date,
    end_date=end_date,
    # system_time=system_time,
    # polling_count=10,
    # polling_interval=1
)

In [ ]:
res.pivot(index='period', columns='id', values='strike')

In [ ]:
res

In [ ]:
hist_curves = res.pivot(index='period', columns='id', values='strike')

In [ ]:
hist_curves.dropna()

In [ ]:
bbg_data = d.query(f'TradeType in {bbg_instruments}')

In [ ]:
bbg_history = bq.bdh(
    securities=bbg_data['cticker'].drop_duplicates(),
    fields=['yld_ytm_mid'],
    start_date=start_date,
    end_date=end_date,
)

In [ ]:
yld = bbg_history.pivot(index='date', columns='security', values='yld_ytm_mid')

In [ ]:
hist_data = pd.concat([yld, hist_curves], axis=1).ffill().dropna()

In [ ]:
for index, group in data.query('Category == "rates"').groupby('Strategy', sort=False):
    break

In [ ]:
tickers = i['cticker'] if i['as_swap']

In [ ]:
hist_data.loc[:, group['cticker'] , group['Description']]

In [ ]:
group['cticker']

In [ ]:
hist_data

In [ ]:
group


In [ ]:
data = get_clarion_positions()

In [ ]:
delta_tol = 100
delta_mul = -1
age='3m'
add_ptf=True
adj_fut=False
stir_as_swap=True

In [ ]:
options = [
    'RatesTreasuryFuturesOption',
    'RatesShortFuturesOption',
]

future_exclusions = [
    'FxForward',
]

spot_map = {
    'rx': 'GTDEM10Y Govt',
    'sfr': 'SOFRRATE Index',
    'ff': 'FEDL01 Index',
    'ed': 'US0003M Index',
    '0e': 'US0003M Index'   
}

curve_map = {
    'USD CBT 30D': 'USD.OIS',
    'EUR ESTER': 'EUR.ESTER',
    'USD SOFR CME 3M': 'USD.SOFR',
    'USD CME 3M': 'USD.3M'
}

delta_map = {
    'RatesTreasuryFutures': -1,
    'RatesTreasuryFuturesOption': -1,
    'RatesShortFutures': -1, 
    'RatesShortFuturesOption': -1
}

field_map = {
    'rx': 'yld_ytm_mid',
    'tu': 'yld_ytm_mid',
    'fv': 'yld_ytm_mid',
    'ty': 'yld_ytm_mid',
    'us': 'yld_ytm_mid',
    'du': 'yld_ytm_mid',
    'oe': 'yld_ytm_mid',
    'rx': 'yld_ytm_mid',
    'ub': 'yld_ytm_mid',
    'ik': 'yld_ytm_mid',
}

In [ ]:
options_tickers = data.query(f'TradeType in {options}').drop_duplicates('Underlying')[['Ticker', 'Underlying']]
options_underlying_tickers = bq.bdp(options_tickers['Ticker'], {'OPT_UNDL_TICKER': 'uticker'}).rename({'security': 'Ticker'}, axis=1)
options_underlying_tickers['uticker'] = options_underlying_tickers['uticker'] + ' Comdty'
options_tickers = options_tickers.merge(options_underlying_tickers, on='Ticker')

In [ ]:
d = data.merge(options_tickers.drop('Ticker', axis=1), on='Underlying', how='left')
d = d.replace('', np.nan)
d['uticker'] = d['uticker']
d['TradeType'] = d['TradeType']
d.loc[~d['TradeType'].isin(options), 'uticker'] = d.loc[~d['TradeType'].isin(options), 'Ticker']
future_tickers = d.query(f'TradeType not in {future_exclusions}')['uticker'].drop_duplicates().dropna()
gen_fut = get_generic_future(future_tickers, bq=bq);
future_tickers = future_tickers.to_frame().assign(
    gticker=gen_fut,
    cticker=[get_bbg_fut_chain_ticker(fut, days=10, yk=None) for fut in gen_fut] if adj_fut else gen_fut,
)
d = d.merge(future_tickers, on='uticker', how='left')

In [ ]:
def get_future_root(ticker):
    if not pd.isnull(ticker):
        return ticker.split(' ')[0].strip()[:-2].lower()
    return np.nan


def get_aged_futures(uticker, gticker):   
    if not pd.isnull(gticker):
        root = uticker.split(' ')[0].strip()[:-2]
        end = gticker.split(' ', 1)[-1].strip()
        n = int(gticker.split(' ')[0].strip()[len(root):])
        if n > 1:
            aged = f"{root}{n-1} {end}"
        else:
            aged = spot_map[root.lower()]
    else:
        aged = np.nan
    return aged


def get_swap_curve(underlying):
    return curve_map.get(underlying, np.nan)


def get_hpstring(
    instrument,
    curve,
    period, 
    ticker,
    root,
    stir_as_swap=True
):
    if instrument in ['RatesTreasuryFutures', 'RatesTreasuryFuturesOption']:
        field = 'YLD_YTM_MID' # field_map.get(root)
        return f'[BBG {ticker} {field}]'
    elif instrument in ['RatesShortFutures', 'RatesShortFuturesOption']:
        if stir_as_swap:
            return f'[s {curve} {period}]'
        return f'(100 - [BBG {ticker}])' if ticker not in spot_map.values() else f'[BBG {ticker}]'
    elif instrument in ['RatesSwap']:
        return f'[s {curve} {period}]' 

In [ ]:
d['rticker'] = d.apply(lambda row: get_future_root(ticker=row['uticker']), axis=1)
d['cticker_aged'] = d.apply(lambda row: get_aged_futures(uticker=row['uticker'], gticker=row['cticker']), axis=1)
d['hpcurve'] = d.apply(lambda row: get_swap_curve(underlying=row['Underlying']), axis=1)
d['hpperiod'] = d.apply(lambda row: parse_swap_periods(start_date=row['StartDate'], end_date=row['MaturityDate']), axis=1)
d['hpperiod_aged'] = d.apply(lambda row: parse_swap_periods(start_date=row['StartDate'], end_date=row['MaturityDate'], age='3m'), axis=1)
# d['delta_rate'] = d.apply(lambda row: row['Delta'] * delta_map.get(row['TradeType'], 1), axis=1)
d['delta_rate'] = d['Delta']

In [ ]:
d['string'] = d.apply(lambda row: get_hpstring(
    instrument=row['TradeType'],
    curve=row['hpcurve'],
    period=row['hpperiod'],
    ticker=row['cticker'],
    root=row['rticker'],
    stir_as_swap=stir_as_swap
), axis=1)

d['string_aged'] = d.apply(lambda row: get_hpstring(
    instrument=row['TradeType'],
    curve=row['hpcurve'],
    period=row['hpperiod_aged'],
    ticker=row['cticker_aged'],
    root=row['rticker'],
    stir_as_swap=stir_as_swap
), axis=1)

In [ ]:
d_ = d.query('Category == "rates"').groupby(['Strategy', 'string', 'string_aged'])['delta_rate'].sum().reset_index()
d_ = d_.loc[~d_['delta_rate'].between(-delta_tol, delta_tol), :]
d_['Strategy'] =  d_['Strategy'].str.split(':', n=1).str[1:].str[0].str.replace(' ', '_').str.lower()
d_['delta_rate'] = d_['delta_rate'] * delta_mul

In [ ]:
bpv = 1e4

parsed = {}
round_ = 1
for index, group in d_.groupby('Strategy'):
    s_base = (group['delta_rate'] / bpv).apply(lambda x: format(x, f'+.{round_}f')) + '*' 
    s = f"{' '.join(s_base + group['string'])}"
    sa = f"{' '.join(s_base + group['string_aged'])}"
    parsed[index] = s
    parsed[f"{index}A"] = sa

parsed = pd.Series(parsed, dtype=object)

if add_ptf:
    ptf = pd.Series(
        [
            ' + '.join(parsed.index[[*range(0, len(parsed), 2)]]),
            ' + '.join(parsed.index[[*range(1, len(parsed), 2)]])
        ],
        index=['ptf', 'ptfA']
    )
    parsed = pd.concat([parsed, ptf])
parsed.to_clipboard(header=False)

In [ ]:
clarion.metrics()["positions_metrics"][:30]

In [ ]:
clarion.price_metrics().sort_values("metricType")[100:]

In [ ]:
# Create a list of metrics in form clarion.metric('label', 'metricType', 'assetClass', {config})
# 'label' here is an arbitrary string that can be used to identify the metric
pv = clarion.metric("pv", "PresentValue", "Any")
strike = clarion.metric("strike", "Strike", "Any")
maturity = clarion.metric("maturity_date", "MaturityDate", "Rates", {"Type": "Bumped"})
start = clarion.metric("start_date", "StartDate", "Rates", {"Type": "Bumped"})

fx_price = clarion.metric("Price", "Price", "Fx", {"Unit": "PercentLocal"})
atm_vol = clarion.metric("AtmVol", "Volatility", "Fx", {"Type": "ATM"})
strike_vol = clarion.metric("StrikeVol", "Volatility", "Fx", {"Type": "Strike"})
carry_t1 = clarion.metric(
    "T+1Carry", "Carry", "Fx", {"Period": "1", "StartBumpDays": "1"}
)


fwd_rate = clarion.metric("fwd_rate", "Forward", "Rates")
strike_vol = clarion.metric(
    "strike_vol", "Volatility", "Rates", {"Type": "Strike", "Period": "Daily"}
)

# m8 = clarion.metric('EqDelta', 'Delta', 'Equity', {'VolSpotDynamic': 'StickyStrike'})
# m9 = clarion.metric('EqPrice', 'Price', 'Equity', {'Unit': 'Local'})

fx_metrics = [pv, fx_price, atm_vol, strike_vol, carry_t1]
rates_metrics = [start, maturity, pv, strike, fwd_rate, strike_vol]
# eq_metrics = [m1,m8,m9]

## Find which products are supported and required parameters

In [ ]:
instrument_types = clarion.price.instrument_types()
instrument_types;

In [ ]:
clarion.price.instrument_type_params("FxVanilla")

### Full Python Code - Rates Swap pricing

In [ ]:
tenors = [
    "o/n",
    "1m",
    "3m",
    "6m",
    "1y",
    "2y",
    "3y",
    "4y",
    "5y",
    "6y",
    "7y",
    "8y",
    "9y",
    "10y",
    "12y",
    "15y",
    "20y",
    "25y",
    "30y",
    "40y",
    "50y",
]

In [ ]:
fwd_starts = [
    None,
    "1m",
    "3m",
    "6m",
    "1y",
    "2y",
    "3y",
    "4y",
    "5y",
    "6y",
    "7y",
    "8y",
    "9y",
    "10y",
]

In [ ]:
offsets = ["1m", "3m", "6m", "1y"]

In [ ]:
swap_floating = "SOFR A"
ccy = "USD"

In [ ]:
trade1 = clarion.instruments.FxVanilla(
    currency_pair="EURUSD", expiry="3M", strike="a", option_type="C"
)

In [ ]:
clarion.price.run(
    fx_metrics,
    trade1.trade,
    start_date="2018-01-01",
    end_date="2022-12-13",
    market_mode="live",
)

In [ ]:
df

In [ ]:
clarion.price.create_trade(
    "RatesSwap",
)

In [ ]:
swap_trades = []
for start in fwd_starts:
    for tenor in tenors:
        trade = clarion.instruments.RatesSwap(
            currency=ccy,
            instrument_name=swap_floating,
            strike="a",
            tenor=tenor,
            start_date=start,
            swap_type="Pay",
            # notional='d20000',
            clearing_house="LCH",
            id=f"{start}x{tenor}" if start else f"spotx{tenor}",
        )
        swap_trades.append(trade.trade)

In [ ]:
end_date = today()
offset = "1y"

In [ ]:
data_list.__len__()

In [ ]:
df = clarion.runner.run(
    rates_metrics,
    swap_trades,
    market_mode="live",
    start_date=end_date - parse_offset(offset),
    end_date=end_date,
    # system_time=system_time,
    # polling_count=10,
    # polling_interval=1
)

In [ ]:
df

In [ ]:
clarion.runner._reqs[0]

In [ ]:
clarion.price.run(

In [ ]:
len(swap_trades)

In [ ]:
res = clarion.price.retrieve(req["KEY"], polling_count=10, polling_interval=1)

In [ ]:
req

In [ ]:
res = clarion.price.retrieve(req["KEY"], polling_count=10)

In [ ]:
res.keys()

In [ ]:
res["TIMEOUT"]

In [ ]:
d = res["DFRAME"]

In [ ]:
res = clarion.price.retrieve(req["KEY"])

In [ ]:
res["DFRAME"]

In [ ]:
def run_multiple();

In [ ]:
swap_curves = res["DFRAME"].sort_values(["period", "start_date", "maturity_date"])
periods = swap_curves.id.str.split("x")
swap_curves = swap_curves.assign(
    start=periods.str[0],
    tenor=periods.str[1],
    date=pd.to_datetime(swap_curves["period"]),
)

In [ ]:
date = today()

In [ ]:
swap_curves.query(f"date=='{date:%Y-%m-%d}'").plot(
    x="tenor",
    y="strike",
    color="start",
    title=swap_floating,
    height=500,
    template="plotly_white",
)

In [ ]:
fra_floating = "LIBOR 3M"
ccy = "USD"

In [ ]:
fra_trades = []
for start in fwd_starts:
    trade = clarion.instruments.RatesFra(
        currency=ccy,
        instrument_name=fra_floating,
        strike="a",
        start_date=start,
        fra_type="Pay",
        # notional='d20000',
        clearing_house="LCH",
        id=f"{start}x3m" if start else f"spotx3m",
    )
    fra_trades.append(trade.trade)

In [ ]:
offsets = ["1m", "3m", "6m", "1y"]

In [ ]:
end_date = today()
offset = "1y"

In [ ]:
response = clarion.price.run(
    rates_metrics,
    fra_trades,
    market_mode="live",
    start_date=f"{end_date - parse_offset(offset):%Y-%m-%d}",
    end_date=f"{end_date:%Y-%m-%d}",
)

In [ ]:
fra_curves = response["DFRAME"].sort_values(["period", "start_date", "maturity_date"])
periods = fra_curves.id.str.split("x")
fra_curves = fra_curves.assign(
    start=periods.str[0], tenor=periods.str[1], date=pd.to_datetime(curves["period"])
)

In [ ]:
fra_curves

In [ ]:
" " + fra_curves.query(f"date=='{date:%Y-%m-%d}'")["start"]

In [ ]:
fra_curves.query(f"date=='{date:%Y-%m-%d}'").plot(
    x="start", y="strike", title=floating, height=500, template="plotly_white"
)

### Full Python Code - Rates swaption pricing

In [ ]:
rates_swaption = clarion.create_trade(
    "swaption_1",
    "ratesSwaption",
    {
        "InstrumentName": "BBSW Q 3M",
        "Currency": "AUD",
        "Expiry": "10-Mar-23",
        "Notional": 100000000,
        "Tenor": "6M",
        "Strike": "0.2",
        "SwaptionType": "Pay",
        "CollateralCurrency": "USD",
        "SettleType": "Physical Cleared LCH",
        "PaySettle": "Spot",
    },
)

response = clarion.price.run(
    rates_metrics,
    rates_swaption,
    start_date="2022-03-01",
    end_date="2022-03-16",
    market_mode="reval",
)
if response["ERROR"]:
    print(response["ERROR"])
    print(response["IS_ERROR"])
    print(response["HAS_ERROR"])
else:
    display(response["DFRAME"])

### Full Python Code - Equity future option pricing

In [ ]:
equity_futOpt = clarion.create_trade(
    "id2",
    "equityFutureOption",
    {
        "InstrumentName": "AS51 SFE",
        "UnderlyingExpiry": "Sep23",
        "ContractType": "OTC",
        "Expiry": "2023-09-21",
        "Strike": "5500",
        "OptionType": "C",
        "ExerciseType": "A",
        "Quantity": 100,
    },
)
response = clarion.price.run(
    eq_metrics,
    equity_futOpt,
    start_date="2022-06-01",
    end_date="2022-09-01",
    market_mode="reval",
)
if response["ERROR"]:
    print(response["ERROR"])
    print(response["IS_ERROR"])
    print(response["HAS_ERROR"])
else:
    display(response["DFRAME"])

## Retrieve Exchange close pricing
#### Exchange close pricing is available back up to **4w history** from todays date

In [ ]:
rates_STIRFut = clarion.create_trade(
    "stir1",
    "ratesShortTermFuture",
    {"InstrumentName": "EURIBOR 3M ICE", "Currency": "EUR", "Tenor": "H23"},
)

In [ ]:
m1 = clarion.metric("FwdRate", "Forward", "Rates")
response = clarion.price.run(
    [m1],
    rates_STIRFut,
    start_date="2022-09-01",
    end_date="2022-09-05",
    market_mode="reval",
    settings={"market_name": "exchange"},
)
response["DFRAME"]